In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import glob
import pandas as pd
from scipy import signal as sn
from scipy.signal import savgol_filter 
from plotly import express as px
plt.style.use('ggplot')


videos=['videos/20250528_170906-003.mp4','videos/20250528_173138.mp4','videos/20250528_173920.mp4','videos/20250528_175822.mp4',
       'videos/C1862.MP4','videos/C1871.MP4','videos/C1872.MP4','videos/C1873.MP4','videos/C1883.MP4','videos/C1888.MP4','videos/C1889.MP4',
       'videos/C1896.MP4','videos/C1897.MP4','videos/C1899.MP4','videos/C1906.MP4','videos/C1907.MP4',
       'videos/C1910.MP4','videos/C1915.MP4','videos/C1917.MP4','videos/C1918.MP4','videos/C1923.MP4']
paths=[f"frames_video{i+1}/*" for i in range(len(videos))]

In [ ]:
k=19
path_list=glob.glob(paths[k])
cam = cv.VideoCapture(videos[k])
fps = cam.get(cv.CAP_PROP_FPS)
print(fps)
cam.release()
dt=1/fps
t=np.array([dt*x for x in range(len(path_list))])

L,a,b=np.zeros(shape=(3,t.shape[0]))
L_in,a_in,b_in=0,0,0

p1 = path_list[7750]
p2 = path_list[21750]

img1 = cv.imread(p1,cv.IMREAD_GRAYSCALE)
img1 = img1[1193:1819 , 1274:2100]

img2= cv.imread(p2,cv.IMREAD_GRAYSCALE)
img2 = img2[1193:1819 , 1274:2100]

delta = (img1 - img2).astype(np.float64)

arr_new = np.where(delta > 235, 0, delta)       #si soy completamente sincero varias de estas ultimas 
arr_new2 = np.where(arr_new > 30, 255, arr_new) #lineas fueron random bullshit go, ni tengo idea de 
arr_new2[arr_new2 < 30] = 0                     #si esto hace algo o no
mask = (arr_new2 == 255)
size = np.sum(mask)
size




videocap = cv.VideoCapture(videos[k])
total_frames = int(videocap.get(cv.CAP_PROP_FRAME_COUNT))
frame_interval = int(fps)  # One frame every second

print(f"FPS: {fps}, Total frames: {total_frames}")

frame_range = range(0, total_frames, frame_interval)

for j, frame_num in enumerate(frame_range) :
    videocap.set(cv.CAP_PROP_POS_FRAMES, frame_num)
    success, frame = videocap.read()
    if not success:
        break


videocap.release()
print("Done reading frames.")



for j,i in enumerate(path_list):
    img = cv.imread(i,cv.IMREAD_COLOR_RGB)
    img = cv.cvtColor(img, cv.COLOR_RGB2LAB) 
    img = img[1193:1819 , 1274:2100]
    
    L_in,a_in,b_in=0,0,0
    output = np.zeros_like(img)         # Todo negro
    output[mask] = img[mask]            # Copiar solo donde mask es True

    L_in += np.sum(img[:,:,0])/size
    a_in += np.sum(img[:,:,1])/size
    b_in += np.sum(img[:,:,2])/size
    
    L[j] = L_in
    a[j] = a_in
    b[j] = b_in

df_video20=pd.DataFrame({'t':t,'L':L,'a':a,'b':b})
df_video20.to_csv('procesados/video20_Lab_sliced.csv',index=False)
plt.plot(df_video20[['L']])